In [1]:
# 📚 Import thư viện
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mobilenet_preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# ----- Cấu hình -----
IMG_SIZE = 224
BATCH_SIZE = 32
DATA_PATH = '../../../Bai_4/data'
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
DARK_PATH = '../../../Bai_4/data_dark'
ALGORITHM_PATH = '../../../Bai_4/data_algorithm'
TRAIN_DARK_PATH = os.path.join(DARK_PATH, 'train')
TEST_DARK_PATH = os.path.join(DARK_PATH, 'test')
TRAIN_ALGORITHM_PATH = os.path.join(ALGORITHM_PATH, 'train')
TEST_ALGORITHM_PATH = os.path.join(ALGORITHM_PATH, 'test')

# Tạo các thư mục nếu chưa tồn tại
os.makedirs(TRAIN_DARK_PATH, exist_ok=True)
os.makedirs(TEST_DARK_PATH, exist_ok=True)
os.makedirs(TRAIN_ALGORITHM_PATH, exist_ok=True)
os.makedirs(TEST_ALGORITHM_PATH, exist_ok=True)

# ----- Hàm tạo ảnh tối -----
def create_dark_image(image, brightness_range=[0.1, 0.1]):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hsv[..., 2] = hsv[..., 2] * np.random.uniform(brightness_range[0], brightness_range[1])
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

# ----- Hàm tăng cường sáng -----
def adaptive_augmentation(image, T1=50, T2=100, gamma_low=0.5, gamma_mid=0.8):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    mean_intensity = np.mean(gray)
    if mean_intensity < T1:
        image = gamma_correction(image, gamma_low)
    elif T1 <= mean_intensity < T2:
        image = gamma_correction(image, gamma_mid)
    else:
        image = contrast_stretching(image)
    return image

def gamma_correction(image, gamma):
    table = np.array([((i / 255.0) ** gamma) * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

def contrast_stretching(image):
    stretched = np.zeros_like(image)
    for c in range(3):
        min_val = np.min(image[:, :, c])
        max_val = np.max(image[:, :, c])
        stretched[:, :, c] = ((image[:, :, c] - min_val) * 255.0 / (max_val - min_val + 1e-6))
    return stretched.astype(np.uint8)

# ----- Hàm xử lý ảnh -----
def process_images_for_train_and_test(path, dark_path, algorithm_path):
    for class_name in os.listdir(path):
        class_path = os.path.join(path, class_name)
        if not os.path.isdir(class_path):
            continue

        # Tạo thư mục cho ảnh tối và ảnh sau thuật toán
        dark_class_path = os.path.join(dark_path, class_name)
        algorithm_class_path = os.path.join(algorithm_path, class_name)
        
        os.makedirs(dark_class_path, exist_ok=True)
        os.makedirs(algorithm_class_path, exist_ok=True)

        for file_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
            input_file = os.path.join(class_path, file_name)
            
            # Đọc ảnh, resize và chuyển sang RGB
            img = cv2.imread(input_file)
            if img is None:
                continue
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # 1. Tạo ảnh tối và lưu
            dark_img = create_dark_image(img_rgb)
            dark_file_path = os.path.join(dark_class_path, file_name)
            cv2.imwrite(dark_file_path, cv2.cvtColor(dark_img, cv2.COLOR_RGB2BGR))

            # 2. Áp dụng thuật toán tăng cường sáng và lưu ảnh
            enhanced_img = adaptive_augmentation(dark_img)
            enhanced_file_path = os.path.join(algorithm_class_path, file_name)
            cv2.imwrite(enhanced_file_path, cv2.cvtColor(enhanced_img, cv2.COLOR_RGB2BGR))

# Gọi hàm để xử lý ảnh trong tập train và test
process_images_for_train_and_test(TRAIN_PATH, TRAIN_DARK_PATH, TRAIN_ALGORITHM_PATH)
process_images_for_train_and_test(TEST_PATH, TEST_DARK_PATH, TEST_ALGORITHM_PATH)

Processing disgust: 100%|██████████| 111/111 [00:00<00:00, 566.51it/s]


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

IMG_SIZE = 224
BATCH_SIZE = 32

# Dùng ImageDataGenerator cho train
train_datagen = ImageDataGenerator(
    preprocessing_function=mobilenet_preprocess_input,
)

train_generator = train_datagen.flow_from_directory(
    '../../../Bai_4/data_algorithm/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# Load toàn bộ test ảnh
test_datagen = ImageDataGenerator(
    preprocessing_function=mobilenet_preprocess_input,
)

full_test_generator = test_datagen.flow_from_directory(
    '../../../Bai_4/data_algorithm/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# Tải toàn bộ ảnh test ra numpy array
test_images = []
test_labels = []

for i in range(len(full_test_generator)):
    x, y = full_test_generator[i]
    test_images.append(x[0])
    test_labels.append(y[0])

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Chia đôi test thành val và test
X_val, X_test, y_val, y_test = train_test_split(
    test_images, test_labels, test_size=0.5, random_state=42, stratify=test_labels
)

# Tạo generator từ numpy
val_generator = test_datagen.flow(
    X_val, y_val, batch_size=1, shuffle=False
)

test_generator = test_datagen.flow(
    X_val, y_val, batch_size=1, shuffle=False
)
print(f"Tổng số ảnh train: {train_generator.n}")
print(f"Tổng số ảnh val: {val_generator.n}")
print(f"Tổng số ảnh test: {test_generator.n}")

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Tổng số ảnh train: 28709
Tổng số ảnh val: 3589
Tổng số ảnh test: 3589
